# Probabilistic Reconciliation Methods Comparison

This notebook compares the different probabilistic reconciliation methods available in HierarchicalForecast:
- **Normality**: Gaussian-based, parametric approach
- **Bootstrap**: Non-parametric residual resampling
- **PERMBU**: Empirical copula-based with rank permutation
- **Conformal**: Distribution-free with coverage guarantees under exchangeability

In [ ]:
# Install dependencies if needed
# !pip install hierarchicalforecast statsforecast

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from statsforecast.models import AutoARIMA
from statsforecast.core import StatsForecast

from hierarchicalforecast.utils import aggregate
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, MinTrace

## 1. Load and Prepare Data

We use the Australian Tourism dataset for this example.

In [ ]:
# Load tourism data
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df = Y_df[['Country', 'Region', 'State', 'Purpose', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df.head()

In [ ]:
# Define hierarchical structure
spec = [
    ['Country'],
    ['Country', 'State'],
    ['Country', 'State', 'Region'],
]

Y_df, S_df, tags = aggregate(df=Y_df, spec=spec)
print(f"Hierarchy has {S_df.shape[0]} series ({S_df.shape[1]} bottom-level)")

In [ ]:
# Train/test split
Y_test_df = Y_df.groupby('unique_id').tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

print(f"Training: {len(Y_train_df)} observations")
print(f"Testing: {len(Y_test_df)} observations")

## 2. Compute Base Forecasts

In [ ]:
# Fit base forecaster
fcst = StatsForecast(
    models=[AutoARIMA(season_length=4)],
    freq='QE',
    n_jobs=-1
)

# Get forecasts and fitted values for probabilistic methods
Y_hat_df = fcst.forecast(df=Y_train_df, h=8, fitted=True)
Y_fitted_df = fcst.forecast_fitted_values()

## 3. Probabilistic Reconciliation Methods

### Method Comparison Table

| Method | Distributional Assumptions | Residual Usage | Reconciliation Timing | Coverage Guarantee | Hierarchy Requirement |
|--------|---------------------------|----------------|----------------------|-------------------|----------------------|
| **Normality** | Gaussian errors | Computes covariance from residuals | N/A (analytical) | Asymptotic | Any |
| **Bootstrap** | None | Block resamples raw residuals, then reconciles | After perturbation | Asymptotic | Any |
| **PERMBU** | None (uses empirical marginals) | Rank permutations for copula | Bottom-up aggregation | Asymptotic | Strictly hierarchical only |
| **Conformal** | Exchangeability | Scores from reconciled residuals | Before perturbation | Finite-sample: $(1-\alpha) \cdot n/(n+1)$ | Any |

### Key Technical Differences

**Bootstrap vs Conformal residual handling:**
- **Bootstrap**: Adds residual blocks to *raw* forecasts, then applies reconciliation (`SP @ (y_hat + residuals)`)
- **Conformal**: Computes scores from *reconciled* forecasts, adds to reconciled predictions (`y_rec + scores`)

In [ ]:
# Reconcile with different probabilistic methods
reconcilers = [
    MinTrace(method='ols'),
]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)

In [ ]:
# Normality-based intervals
Y_rec_normality = hrec.reconcile(
    Y_hat_df=Y_hat_df,
    Y_df=Y_fitted_df,
    S_df=S_df,
    tags=tags,
    level=[90],
    intervals_method='normality'
)
print("Normality reconciliation complete")

In [ ]:
# Bootstrap-based intervals
Y_rec_bootstrap = hrec.reconcile(
    Y_hat_df=Y_hat_df,
    Y_df=Y_fitted_df,
    S_df=S_df,
    tags=tags,
    level=[90],
    intervals_method='bootstrap'
)
print("Bootstrap reconciliation complete")

In [ ]:
# Conformal-based intervals
Y_rec_conformal = hrec.reconcile(
    Y_hat_df=Y_hat_df,
    Y_df=Y_fitted_df,
    S_df=S_df,
    tags=tags,
    level=[90],
    intervals_method='conformal'
)
print("Conformal reconciliation complete")

## 4. Visualize Prediction Intervals

In [ ]:
def plot_intervals(series_id, ax, rec_df, method_name, color):
    """Helper to plot prediction intervals for a series."""
    df = rec_df[rec_df['unique_id'] == series_id].copy()
    
    col_mean = 'AutoARIMA/MinTrace_method-ols'
    col_lo = f'{col_mean}-lo-90'
    col_hi = f'{col_mean}-hi-90'
    
    ax.fill_between(df['ds'], df[col_lo], df[col_hi], alpha=0.3, color=color, label=f'{method_name} 90% PI')
    ax.plot(df['ds'], df[col_mean], color=color, linewidth=2)

# Plot comparison for top-level series
series_id = 'Australia'

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

plot_intervals(series_id, axes[0], Y_rec_normality, 'Normality', 'blue')
axes[0].set_title('Normality')
axes[0].legend()

plot_intervals(series_id, axes[1], Y_rec_bootstrap, 'Bootstrap', 'green')
axes[1].set_title('Bootstrap')
axes[1].legend()

plot_intervals(series_id, axes[2], Y_rec_conformal, 'Conformal', 'orange')
axes[2].set_title('Conformal')
axes[2].legend()

fig.suptitle(f'Prediction Intervals Comparison: {series_id}')
plt.tight_layout()
plt.show()

## 5. Method Pros and Cons

### Normality
**Pros:**
- Fast computation using closed-form solutions
- Well-understood theoretical properties
- Works with any reconciliation method and hierarchy structure
- Supports different covariance estimators (`ols`, `wls_var`, `mint_shrink`, etc.)

**Cons:**
- Assumes Gaussian distribution of errors
- May underestimate uncertainty for heavy-tailed or skewed distributions
- Covariance estimation can be unstable with limited data

---

### Bootstrap
**Pros:**
- Non-parametric: no distributional assumptions required
- Captures empirical error distribution shape (skewness, heavy tails)
- Preserves temporal correlation through block resampling
- Works with any hierarchy structure

**Cons:**
- Requires sufficient historical residuals (at least horizon + some buffer)
- Computationally more expensive than Normality
- Coverage is asymptotic (no finite-sample guarantees)

---

### PERMBU
**Pros:**
- Preserves empirical dependencies using copula-based approach
- Respects marginal distributions at each level
- Captures complex cross-series dependencies

**Cons:**
- **Only works with strictly hierarchical structures** (not grouped hierarchies)
- Computationally intensive for large hierarchies
- Requires careful handling of the permutation ordering

---

### Conformal
**Pros:**
- Distribution-free: no parametric assumptions required
- Finite-sample coverage guarantee under exchangeability
- Simple interpretation: intervals based on empirical quantiles of scores
- Works with any hierarchy structure

**Cons:**
- **Requires proper calibration set**: Coverage guarantees assume the calibration data is independent from training data. When using in-sample residuals (fitted values from the same data used to train the model), this assumption is violated, potentially leading to overly optimistic (narrow) intervals
- Exchangeability assumption may not hold for time series with trends or structural breaks
- Coverage guarantees are marginal (per-series), not simultaneous across the hierarchy
- May produce wider intervals than well-specified parametric methods

**⚠️ Important Caveat for Conformal:**
In this example (and the default HierarchicalForecast API), we use in-sample fitted values as the calibration set. This is convenient but technically violates the split conformal prediction framework. For rigorous coverage guarantees, you should use a held-out validation set that was not used for model training.

## 6. Recommendations

| Scenario | Recommended Method | Notes |
|----------|-------------------|-------|
| Quick analysis, errors appear Gaussian | **Normality** | Fastest, well-understood |
| Unknown error distribution, general use | **Bootstrap** | Safe default, no assumptions |
| Strict hierarchies, need correlation preservation | **PERMBU** | Best for capturing dependencies |
| Have proper held-out calibration set | **Conformal** | Valid finite-sample guarantees |
| Using in-sample residuals, want simplicity | **Bootstrap** or **Normality** | Conformal caveats apply with in-sample data |

### Decision Flowchart

1. **Is your hierarchy strictly hierarchical (no cross-classifications)?**
   - Yes → Consider PERMBU if you need correlation preservation
   - No → Use Normality, Bootstrap, or Conformal

2. **Do you have a proper held-out calibration set?**
   - Yes → Conformal provides finite-sample coverage guarantees
   - No (using in-sample) → Bootstrap or Normality are more appropriate

3. **Do your residuals appear Gaussian?**
   - Yes → Normality is fast and efficient
   - No / Unknown → Bootstrap adapts to any distribution